In [297]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head(5)

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


In [298]:
# 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)
top_channels = df["channelTitle"].value_counts(ascending=False).index.tolist() # value_counts로 인덱스로 들어간 채널명을 가져와 리스트로 변경
top_channels
# 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라
df.query('dislikes > likes')['channelTitle'].unique() # 쿼리 함수, 고유값찾기
# 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라
change = df.groupby('channelId')['channelTitle'].nunique() # 채널id별 채널명의 고유값 갯수 가져오고 채널id가 2개 이상인 값 추출, 그 값의 length
change_channel = change[change > 1]
len(change_channel)
# 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?
df['trending_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # sql에 TO_DATE와 유사한 기능 yyyy면 y가 대문자
sunday_df = df[df['trending_date'].dt.weekday == 6] # dt.weekday()는 0이 월요일, 6이 일요일 tending_date가 dt.weekday 6 인 값들
sundday_categoryId = sunday_df['categoryId'].value_counts() # 6인 값들의 채널id들
sundday_categoryId.head(1)
# 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라
df['tranding_date'] = pd.to_datetime(df['trending_date2'], format='%Y-%m-%d') # pd.to_datetime으로 변환
df['day_name'] = df['tranding_date'].dt.day_name() # dt.day_name으로 요일이름 가져오기
group = df.groupby(['day_name', 'categoryId']).size().reset_index(name="count")# 요일이름 채널 아이디의 사이즈 출력하고 인덱스로 바꾼뒤 인덱스 초기화 후 count 컬럼 생성
pivot_table = group.pivot(index='day_name', columns='categoryId', values='count') # 피봇으로 인덱스,컬럼,값 지정
pivot_table.transpose() # 채널id와 요일이름 위치변경
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)
zero = df.loc[df['view_count'] != 0] # 조회수가 0인 값 제외
comment = zero['comment_count'] / zero['view_count'] # 비율 계산 
best_index = comment.sort_values(ascending=False).index[0] # view카운트 대비 댓글수가 가장 높은 영상의 채널의 인덱스
result = df.loc[best_index, 'title'] # 해당 인덱스의 제목 출력
result
# 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)
zero = df.loc[df['view_count'] != 0]
comment = zero['comment_count'] / zero['view_count']
ratio = comment[comment != 0] # 비율 계산 후 0이 아닌값
min_ratio = ratio.idxmin() # 0이 아닌 값중에 제일 낮은 값 idxmin() 인덱스 중 최소값
title = zero.loc[min_ratio, 'title'] 
title
# like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)
zero = df.loc[(df['likes'] != 0) & (df['dislikes'] != 0),['likes', 'dislikes', 'title']] # loc의 and 연산 0제외
ratio = zero['dislikes'] / zero['likes'] # 비율 계산
min_likes = ratio.idxmin() # 인덱스 최소값
title = zero.loc[min_likes,'title'] # 인덱스 최소값 영상의 제목
title
# 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)
best_create = df['channelId'].value_counts().index[0]  # 채널id가 가장 많은 인덱스
title = df.loc[df['channelId']==best_create,'channelTitle'].unique() # id가 많은, 영사잉 많은 채널 id의 채널 이름 , 중복제거
title
# 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?
best_video = (df[['title','channelTitle']].value_counts() >= 20).sum()
best_video

np.int64(40)